## Prepare training data

In [ ]:
# %%capture cap

%load_ext autoreload
%autoreload 2


import os

os.environ['OMP_NUM_THREADS'] = "6,1"
os.environ["ASE_DFTB_COMMAND"] = "ulimit -s unlimited; /home/dlb/anaconda3/envs/dftb_plumed/bin/dftb+ > PREFIX.out"
os.environ["DFTB_PREFIX"] = "/home/dlb/Downloads/pbc-0-3"

# https://www.plumed.org/doc-v2.7/user-doc/html/_m_e_t_a_d.html
# https://github.com/dftbplus/dftbplus/issues/1064
# try with numpy 1.22　
# https://gitlab.com/Sucerquia/ase-plumed_tutorial

from ase.calculators.lj import LennardJones
# from ase.calculators.plumed import Plumed
from runner.plumed import Plumed
# from plumed import Plumed
from ase.constraints import FixedPlane
from ase.io import read
from ase import units

from ase.geometry.analysis import Analysis
from ase.constraints import FixAtoms, FixBondLengths

# import plumed

from ase.calculators.dftb import Dftb



ps = 1000 * units.fs

setup = [f"UNITS LENGTH=A TIME={1/ps} ENERGY={units.mol/units.kJ}",
        #  "d: DISTANCE ATOMS=1,2 ",
        "at8: FIXEDATOM AT=-0.3652684296711638,5.59639757164289,11.089411052917308",
        "at6: FIXEDATOM AT=-0.5388786945959876,6.471914514609469,10.07028636148657",
        "c1: CENTER ATOMS=at8,at6",
        "c2: CENTER ATOMS=67,69,73,77,79,83",
        "c3: CENTER ATOMS=7,9,11,13,15,17",
        "c4: CENTER ATOMS=79,83",
         "phi: TORSION ATOMS=c1,c2,c3,c4",
         "metad: METAD ARG=phi SIGMA=1.10 HEIGHT=15.0 BIASFACTOR=5 TEMP=300.0 PACE=1", #GRID_MIN=-pi GRID_MAX=pi GRID_BIN=1500
         # "ARG=d SIGMA=0.20,0.20 HEIGHT=1.20 BIASFACTOR=5 TEMP=300.0 PACE=500",
         # "GRID_MIN=0,0 GRID_MAX=1.0,1.0 GRID_BIN=150,150",
         "PRINT ARG=phi STRIDE=1 FILE=../results/test/plumed/COLVAR",
         # "CALC_RCT ",
         # "RCT_USTRIDE=10",
         # "...",
         "FLUSH STRIDE=1000"]


atoms = read('../data/structures/new_systems/ktu_002.cif')


# fix_atoms = FixAtoms(indices=[atom.index for atom in atoms if atom.symbol=='N' or atom.symbol=='Si' or atom.symbol=='O'])
# # # ch_bonds = Analysis(atoms).get_bonds("C", "H")[0]
# # # fix_bond_lengths = FixBondLengths(ch_bonds)
# # # atoms.set_constraint([fix_atoms, fix_bond_lengths])
# atoms.set_constraint(fix_atoms)

calc_base = Dftb(atoms=atoms,
            label='crystal',
            kpts=(1,1,1)
            )

# atoms.calc = Plumed(calc=calc_base,
#                     input=setup,
#                     timestep=timestep,
#                     atoms=atoms,
#                     kT=0.1,
#                     log='../results/test/plumed/plumed_log.log')

from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin

atoms.calc = calc_base

# dyn = Langevin(atoms, 0.1, temperature_K=0.1/units.kB, friction=0.1,
#                fixcm=True, trajectory='../results/test/machine_learning/dftb_500.xyz')
# We want to run MD with constant energy using the VelocityVerlet algorithm.
dyn = VelocityVerlet(atoms, dt=0.5*units.fs, trajectory='../results/test/machine_learning/dftb_500.traj')  
# https://wiki.fysik.dtu.dk/ase/ase/md.html

dyn.run(500)


In [ ]:
from ase.io import read, write
from ase.visualize import view

traj = read('../results/test/machine_learning/dftb_500.traj', index=":")
# traj = read('/home/dlb/Downloads/UnbiasMD.xyz', index=":")
# write("test/plumed/biased_MTD.xyz", traj, format="extxyz")
# traj_biased = read('test/plumed/biased_MTD.xyz', index=":")
# traj_unbiased = read('test/plumed/.xyz', index=":")

view(traj)
# print(traj[200].get_potential_energy())
# print(traj[200].get_forces()[33])

In [ ]:
# check if correct forces were written into traj file
# https://github.com/qsnake/ase/blob/master/ase/calculators/singlepoint.py
# atoms = read('../data/structures/new_systems/ktu_002.cif')

from ase.calculators.singlepoint import SinglePointCalculator

snap = traj[200].copy()

calc_base = Dftb(atoms=atoms,
            label='crystal',
            kpts=(1,1,1)
            )

snap.calc = calc_base

print(snap.get_potential_energy())
print(snap.get_forces()[33])

## Train GP model

In [ ]:
from ase.visualize import view
from ase import io

import numpy as np

traj_md = io.read("../results/test/machine_learning/dftb_500.traj", index=":")
energies_md = np.zeros(len(traj_md) )
forces_md = np.zeros( (len(traj_md), len(traj_md[0]), 3 ) )
for i, snap in enumerate(traj_md):
    energies_md[i] = snap.get_potential_energy()
    forces_md[i] = snap.get_forces()
# forces_md = np.load("../../data/xtb_md/forces_xtb_md.npy")
# energies_md = np.load("../../data/xtb_md/energies_xtb_md.npy")

# Training data:
# training_indices = np.sort(  np.random.choice(np.arange(0,10_000), 1000, replace=False) )
training_indices = np.sort(  np.arange(0, 20, 3) )  
traj_train = [traj_md[i] for i in training_indices]
energies_train = energies_md[training_indices]
forces_train = forces_md[training_indices]
train_data = {'trajectory': traj_train, 'energies': energies_train, 'forces': forces_train}

#Test data:
# test_indices = np.sort(  np.random.choice(np.arange(0,92795), 200, replace=False) ) 
test_indices = np.sort(  np.arange(0,10,2) ) 
traj_test = [traj_md[i] for i in test_indices]
energies_test = energies_md[test_indices]
forces_test = forces_md[test_indices]
test_data = {'trajectory': traj_test, 'energies': energies_test, 'forces': forces_test}

data_units = "ev_angstrom" # standard ase units

In [ ]:
# using with fande environment

import sys
sys.path.append("../../fande") # Adds higher directory to python modules path.
# sys.path.append("..") # Adds higher directory to python modules path.


hparams = {
    'dtype' : 'float64',
    'device' : 'gpu'
}

# Descriptors parameters:
soap_params = {
    'species': ["H", "C"],
    'periodic': True,
    'rcut': 6.0,
    'sigma': 0.5,
    'nmax': 5,
    'lmax': 5,
    'average': "off",
    'crossover': True,
    'dtype': "float64",
    'n_jobs': 10,
    'sparse': False,
    'positions': [7, 11, 15]
}



In [ ]:
%load_ext autoreload
%autoreload 2

from fande.data import FandeDataModuleASE
import matplotlib.pyplot as plt

fdm = FandeDataModuleASE(train_data, test_data, hparams, units=data_units)

print(fdm.forces_train_norm.shape)
plt.plot(fdm.forces_train[:,49,0])

In [ ]:
%%time

centers_positions = [15,16,17]
derivatives_positions = [49,50]

soap_grad_array_train, DX_train, train_grad_info = fdm.calculate_invariants_librascal(soap_params, centers_positions, derivatives_positions)
# print(fdm.train_DX.shape)
# print(fdm.test_DX.shape)

In [ ]:
fdm.forces_train.shape

In [ ]:
flatten_forces_sub = fdm.forces_train[:, derivatives_positions, :].flatten()

In [ ]:
n_atoms = 264

In [ ]:
a = train_grad_info[:,1]
b = train_grad_info[:,2]


indices_sub = np.where(
    np.in1d(a%n_atoms, centers_positions) & 
    np.in1d(b%n_atoms, derivatives_positions))[0]

In [ ]:
indices_sub

In [ ]:
##

snapshot_indices = np.unique( train_grad_info[indices_sub][:,0] )
atoms_indices = train_grad_info[indices_sub][2]%n_atoms

der_slice = np.s_[derivatives_positions]

fdm.forces_train[snapshot_indices][:,derivatives_positions,:].flatten().shape

In [ ]:
soap_grad_array_train[indices_sub].shape

In [ ]:
fdm.forces_train[:,atoms_indices,:].shape

In [ ]:
atoms_indices

In [ ]:
DX = np.stack( (soap_grad_array_train[3*indices_sub], soap_grad_array_train[3*indices_sub+1], soap_grad_array_train[3*indices_sub+2]), axis=1 )

In [ ]:
DX.shape

In [ ]:
centers_ind = np.where( train_grad_info[:,1] in [15]) )[0]

# np.where( train_grad_info[:,2] == 49 )[0]

In [ ]:
train_grad_info

In [ ]:
train_grad_info[1364]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_f[:,0])

fdm.train_F.shape

In [ ]:
import torch

DX_train_t = torch.tensor(DX_train)
F_train_t = torch.tensor(F_train)

print(torch.flatten(DX_train_t, start_dim=0, end_dim=1).shape)
print(torch.flatten(F_train_t, start_dim=0, end_dim=1).shape)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(DX_train[0,0,:])
plt.plot(DX_train[0,1,:])
plt.plot(DX_train[0,2,:])

In [ ]:
plt.plot(DX_train[12,0,:])
plt.plot(DX_train[12,1,:])
plt.plot(DX_train[12,2,:])

In [ ]:
DX_train.shape

In [ ]:
F_train.shape

In [ ]:
forces_train.shape

In [ ]:
train_DX.shape

## Train Model

In [ ]:
from fande.models import ModelForces, ModelEnergies, MyCallbacks
from torch.utils.data import DataLoader, TensorDataset, random_split
from pytorch_lightning import Trainer, seed_everything
import numpy as np
# seed_everything(42, workers=True)

train_DX = fdm.train_DX
train_F = fdm.train_F
test_DX = fdm.test_DX
test_F = fdm.test_F

training_random_samples = 300
num_epochs = 700
learning_rate = 0.01

# ind_slice = np.sort( np.concatenate( 
#     ( np.arange(0,4800), np.arange(11*4800,12*4800), np.random.choice(np.arange(4800,59200), 300, replace=False) ) 
#     ) )

ind_slice = np.sort(  np.random.choice(np.arange(0,train_F.shape[0]), training_random_samples, replace=False) ) 
# ind_slice = np.sort(  np.arange(0,2550) ) 
# ind_slice = np.sort(  np.arange(0,train_F.shape[0]) ) 


train_dataset = TensorDataset(train_DX[ind_slice], train_F[ind_slice])
train_loader = DataLoader(train_dataset, batch_size=100_000)

model_f = ModelForces(train_DX[ind_slice], train_F[ind_slice], hparams, learning_rate)

trainer_f = Trainer(
    gpus=1, 
    max_epochs=num_epochs, 
    precision=64,
    weights_summary='full', 
    callbacks=[MyCallbacks()])

trainer_f.fit(model_f, train_loader)

In [ ]:
train_F.shape[0]

## Predictions on test data

In [ ]:
from fande.predict import PredictorASE

test_X = fdm.test_X
test_DX = fdm.test_DX
test_E = fdm.test_E
test_F = fdm.test_F

model_e = None
trainer_e = None


predictor = PredictorASE(
            fdm,
            model_e,
            model_f,
            trainer_e,
            trainer_f,
            hparams,
            soap_params
)


predictor.predict_and_plot_forces()

In [ ]:
print(fdm.test_DX.shape)
print(fdm.test_F.shape)

## Testing area

In [ ]:
import os
import ipyparallel as ipp

cluster = ipp.Cluster(n=4)
with cluster as rc:
    ar = rc[:].apply_async(os.getpid)
    pid_map = ar.get_dict()

In [ ]:
import ipyparallel as ipp
rc = ipp.Cluster(n=4).start_and_connect_sync()

## Run simulations